## SMS Spam 분류

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv'

In [2]:
df = pd.read_csv(url, encoding='latin1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


- 데이터 전처리

In [3]:
# Selection
df = df[['v1','v2']]

In [5]:
# 결측치 확인
df.isna().sum().sum()

0

In [7]:
# 중복 데이터 확인
print(df.shape, df.v2.nunique())

(5572, 2) 5169


In [8]:
# 중복 데이터 제거
df.drop_duplicates(subset=['v2'], inplace=True)
df.shape

(5169, 2)

In [9]:
# Label Encoding, ['ham', 'spam'] --> [0, 1]
df.v1 = df.v1.replace(['ham', 'spam'], [0, 1])
df.head(3)

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...


In [10]:
# Ham/Spam 분포
df.v1.value_counts()

0    4516
1     653
Name: v1, dtype: int64

- 텍스트 전처리

In [11]:
# 구둣점, 숫자 제거
df.v2 = df.v2.str.replace('[^A-Za-z]', ' ', regex=True)
df.v2[0]

'Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat   '

- 데이터셋 분리

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.v2.values, df.v1.values, stratify=df.v1.values, test_size=0.2, random_state=2023
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4135,), (1034,), (4135,), (1034,))

- Text Encoding

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')

In [17]:
# 이렇게 하면 절대로 안됨.
# train/test dataset에서 사용되는 단어가 다름
X_train_cv = cvect.fit_transform(X_train)
X_test_cv = cvect.fit_transform(X_test)
X_train_cv.shape, X_test_cv.shape

((4135, 6494), (1034, 2866))

In [18]:
# 변환된 단어 세트가 train과 test가 동일해야 함
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((4135, 6494), (1034, 6494))

- 학습 및 평가

In [37]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2023)

In [38]:
lrc.fit(X_train_cv, y_train)

LogisticRegression(random_state=2023)

In [39]:
lrc.score(X_test_cv, y_test)

0.9709864603481625

- Bigram

In [23]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((4135, 28822), (1034, 28822))

In [24]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_cv2, y_train)
lrc.score(X_test_cv2, y_test)

0.9680851063829787

- TfidfVectorizer

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
tvect.fit(X_train)
X_train_tv = tvect.transform(X_train)
X_test_tv = tvect.transform(X_test)
X_train_tv.shape, X_test_tv.shape

((4135, 6494), (1034, 6494))

In [26]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_tv, y_train)
lrc.score(X_test_tv, y_test)

0.9458413926499033

- TfidfVectorizer + Bigram

In [28]:
tvect2 = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect2.fit(X_train)
X_train_tv2 = tvect2.transform(X_train)
X_test_tv2 = tvect2.transform(X_test)
X_train_tv2.shape, X_test_tv2.shape

((4135, 28822), (1034, 28822))

In [30]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_tv2, y_train)
lrc.score(X_test_tv2, y_test)

0.9410058027079303

- 실제 데이터로 검증

In [35]:
df = pd.read_csv(url, encoding='latin1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [36]:
sms = [df.v2[0], df.v2[2]]
sms

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]

In [40]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_cv, y_train)

LogisticRegression(random_state=2023)

In [41]:
# 텍스트 전처리, 숫자와 구둣점 제거
import re
sms = map(lambda x: re.sub('[^A-Za-z]', ' ', x), sms)

In [42]:
# feature 변환
sms_cv = cvect.transform(sms)
sms_cv.shape

(2, 6494)

In [43]:
# 예측
lrc.predict(sms_cv)

array([0, 1], dtype=int64)

- 베스트 파라메터 찾기

In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [46]:
pipeline = Pipeline([('CVECT', cvect), ('LRC', lrc)])
lrc = LogisticRegression(random_state=2023)
params = {
    'CVECT__ngram_range': [(1,1), (1,2)],
    'LRC__C': [0.1, 1, 10]
}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 15.8 s
Wall time: 5.07 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT',
                                        CountVectorizer(stop_words='english')),
                                       ('LRC',
                                        LogisticRegression(random_state=2023))]),
             param_grid={'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LRC__C': [0.1, 1, 10]},
             scoring='accuracy')

In [47]:
grid_pipe.best_params_

{'CVECT__ngram_range': (1, 1), 'LRC__C': 10}

In [50]:
pipeline = Pipeline([('CVECT', cvect), ('LRC', lrc)])
lrc = LogisticRegression(random_state=2023)
params = {
    'CVECT__ngram_range': [(1,1), (1,2)],
    'LRC__C': [5, 8, 10, 12, 20]
}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 28.7 s
Wall time: 8.8 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT',
                                        CountVectorizer(stop_words='english')),
                                       ('LRC',
                                        LogisticRegression(random_state=2023))]),
             param_grid={'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LRC__C': [5, 8, 10, 12, 20]},
             scoring='accuracy')

In [51]:
grid_pipe.best_params_

{'CVECT__ngram_range': (1, 1), 'LRC__C': 8}

In [52]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.9738878143133463